# Daily Results

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys
import glob

from datetime import datetime
import time

from entrant import Entrant

import csv
import json

In [2]:
projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

In [3]:
EVENTS_DIR = 'events'
CONFIG_DIR = 'config'
SESSIONS_DIR = 'sessions'
GPSDATA_DIR = 'gpsdata'

In [4]:
CONFIG_JSON = 'config.json'
ENTRANTS_CSV = 'entrants.csv'
SESSIONS_JSON = 'sessions.json'

In [5]:
try:
    wsw_full_refresh = int(os.environ['WSW_FULL_REFRESH'])
except:
    wsw_full_refresh = 1
    
verbose = True

## Generic Class

Generic class to ensure that all custom classes are printable

In [6]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)
    
    def logInfo(self, msg):
        print('INFO:', msg)

    def logWarning(self, msg):
        print('WARNING:', msg)

    def logError(self, msg):
        print('ERROR:', msg)

## SpeedRun Class

Class to manage runs by an entrant

In [7]:
class SpeedRun(Printable):
    def __init__(self, courseId, startTime, duration, speed, cog):
        '''Initialise speed run object'''    
        
        self.courseId, self.startTime, self.duration, self.speed, self.cog = courseId, startTime, duration, speed, cog

## Course Class

Class to manage courses - start / end times

In [8]:
class Course(Printable):
    def __init__(self, entrants, courseDate, courseId, courseDict):
        '''Initialise course object'''

        self.entrants = entrants
        self.participants = {}

        self.courseDate = courseDate
        self.courseId = courseId
        self.startTime = courseDict['Start Time']
        self.endTime = courseDict['End Time']
        
        self.minStartTime = '23:59:59'
        self.maxStartTime = '00:00:00'
        self.numRuns = 0
        self.numGt31s = 0


    def getParticipant(self, gt31Ids, gt31Id, gt31Serial):
        '''Get participant from the GT-31 ID and serial'''

        if gt31Id not in gt31Ids:
            self.logWarning('Unrecognised GT-31 ID on {} - {}'.format(self.courseDate, gt31Id))

            entrantId = max(self.entrants.keys()) + 1
            entrantDict = {
                "ID": entrantId,
                "Gender": "?",
                "First Name": "GT-31",
                "Family Name": gt31Id,
                "First Timer": "?",
                "Status": "Unknown",
                "Craft Type": "Unknown",
                "GT31 ID": gt31Id,
                "GT31 SN": gt31Serial,
                "EntryType": "Unknown",
                "UKWA Member": "?",
            }
            #participant = Entrant(entrantId, entrantDict)
            participant = None

            #self.entrants[entrantId] = participant
            #self.participants[entrantId] = participant
            # TODO gt31Id
        else:
            if gt31Ids[gt31Id].gt31SerialNumbers and gt31Serial not in gt31Ids[gt31Id].gt31SerialNumbers:
                self.logWarning('Unrecognised GT-31 SN for {} ({}) on {} - {} vs {}'.format(
                        gt31Ids[gt31Id].getValue('Name'), gt31Ids[gt31Id].getValue('Craft Type'),
                        self.courseDate,
                        gt31Serial, gt31Ids[gt31Id].gt31SerialNumbers))

            participant = gt31Ids[gt31Id]
            self.participants[participant.getValue('ID')] = participant
            
        return participant


    def storeRun(self, participant, startTime, duration, speed, cog):
        '''Store run in memory'''

        if startTime < self.minStartTime:
            self.minStartTime = startTime
        if startTime > self.maxStartTime:
            self.maxStartTime = startTime

        if startTime >= self.startTime and startTime <= self.endTime:
            participant.storeRun(self.courseDate, self.courseId, startTime, duration, speed, cog)

        self.numRuns += 1


    def loadRuns(self, csvPath, gt31Ids):
        '''Read speeds from CSV files'''

        prevGt31Id = None

        with open(csvPath, 'r') as f:
            csvReader = csv.reader(f)
            for values in csvReader:
                # GPSResults does not output a header line
                run, filename, startTime, duration, speed, cog = values

                # Split up record and format the GT-31 details
                try:
                    gt31Id, gt31Serial, fileDate, fileTime = os.path.splitext(filename)[0].split('_')[:4]
                except:
                    self.logError('Problem parsing GT-31 details in run data - "{}"'.format(filename))
                    raise
                gt31Id = gt31Id.upper()

                # Validate each newly encountered GT-31
                if gt31Id != prevGt31Id:
                    participant = self.getParticipant(gt31Ids, gt31Id, gt31Serial)

                # Only store runs for recognised entrants (via GT-31 ID)
                if participant:
                    self.storeRun(participant, startTime, duration, speed, cog)
                
                prevGt31Id = gt31Id

            self.logInfo('{} runs by {} recognised participants down course {} on {} - {} to {}'.format(
                    self.numRuns, len(self.participants), self.courseId, self.courseDate, self.minStartTime, self.maxStartTime))
            
        if self.minStartTime < self.startTime:
            self.logWarning('Runs found before course {} opened on {} - earliest was {}'.format(
                self.courseId, self.courseDate, self.minStartTime))

        if self.maxStartTime > self.endTime:
            self.logWarning('Runs found after course {} closed on {} - latest was {}'.format(
                self.courseId, self.courseDate, self.maxStartTime))

## Session Class

Class to manage sessions

In [9]:
class Session(Printable):
    def __init__(self, entrants, dataPath, configPath):
        '''Initialise session object'''

        self.entrants = entrants
        self.dataPath = dataPath
        self.configPath = configPath

        self.date = os.path.basename(configPath)
        self.year = self.date[:4]

        self.courses = {}


    def loadConfig(self):
        '''Read config from JSON'''

        filename = os.path.join(self.configPath, SESSIONS_JSON)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            courseDict = json.loads(jsonTxt)
            
            for courseId in courseDict:
                # Course ID should be uppercase but just to be sure...
                courseId = courseId.upper()
                if courseId in self.courses:
                    raise ValueError('Duplicate course "{}" for {}'.format(courseId, self.date))
                course = Course(self.entrants, self.date, courseId, courseDict[courseId])
                self.courses[courseId] = course


    def loadRuns(self, gt31Ids):
        '''Read speeds from CSV files'''

        csvPaths = sorted(glob.glob(os.path.join(self.dataPath, '*')))
        
        for csvPath in csvPaths:
            prefix, sessionDate, courseId = os.path.splitext(os.path.basename(csvPath))[0].split('_')
            suffix = os.path.splitext(csvPath)[1]

            # Verify the filename
            if suffix.lower() != '.csv':
                raise ValueError('Invalid file suffix "{}" for {}'.format(suffix, os.path.basename(csvPath)))
            if prefix.upper() != 'GPSDATA':
                raise ValueError('Invalid file prefix "{}" for {}'.format(prefix, os.path.basename(csvPath)))
            if sessionDate != self.date:
                raise ValueError('Invalid file date "{}" for {}'.format(sessionDate, os.path.basename(csvPath)))
            if courseId.upper() not in self.courses:
                raise ValueError('Invalid course "{}" for {}'.format(courseId, os.path.basename(csvPath)))
            
            course = self.courses[courseId.upper()]
            course.loadRuns(csvPath, gt31Ids)

## Event Class

Class to manage events

In [10]:
class Event():
    def __init__(self, path):
        self.path = path
        self.year = int(os.path.basename(path))

        self.entrants = {}
        self.gt31Ids = {}       
        self.sessions = {}


    def loadEntrants(self):
        '''Read entrants from JSON'''

        csvPath = os.path.join(self.path, CONFIG_DIR, ENTRANTS_CSV)
        
        with open(csvPath, 'r') as f:
            csvReader = csv.reader(f)
            headers = next(csvReader)

            for values in csvReader:
                entrant = Entrant(headers, values)
                if entrant.getValue('ID') not in self.entrants:
                    self.entrants[entrant.getValue('ID')] = entrant
                else:
                    raise ValueError('Duplicate entrant ID "{}"'.format(entrant.getValue('ID')))

        if verbose:
            print('{} entrants loaded'.format(len(self.entrants)))


    def indexGt31s(self):
        '''Create GT-31 indices for entrants'''

        for entrantId in self.entrants.keys():
            entrant = self.entrants[entrantId]

            if 'GT31 ID' in entrant.entrantDict:
                for gt31Id in entrant.gt31Ids:
                    self.gt31Ids[gt31Id.upper()] = entrant
        

    def summariseEntrants(self):
        '''Print summary of entrants'''
        
        pass


    def loadSessions(self):
        '''Load all of the event sessions - absence of config will result in an exception (intentional)'''
        
        dataPaths = sorted(glob.glob(os.path.join(self.path, GPSDATA_DIR, '20[0-9][0-9][0-1][0-9][0-3][0-9]')))
        
        for dataPath in dataPaths:
            configPath = os.path.join(self.path, SESSIONS_DIR, os.path.basename(dataPath))
            session = Session(self.entrants, dataPath, configPath)

            session.loadConfig()
            session.loadRuns(self.gt31Ids)
            
            # TODO - session.processRuns()

            self.sessions[session.date] = session
            

    def processEvent(self):
        '''Read entrants from config folder'''

        if verbose:
            print('Processing {}...'.format(self.year))

        self.loadEntrants()  
        self.indexGt31s()
        self.loadSessions()

        if verbose:
            print('All done!\n')

## Process Years

Process all available years

In [11]:
pc1 = time.perf_counter()

In [12]:
# Read main config
filename = os.path.join(projdir, CONFIG_DIR, CONFIG_JSON)
with open(filename, 'r', encoding='utf-8') as f:
    jsonTxt = f.read()
    config = json.loads(jsonTxt)

In [13]:
# Only process the current year (for now)
eventPaths = sorted(glob.glob(os.path.join(projdir, EVENTS_DIR, '20[0-9][0-9]')))
year = datetime.now().year
for eventPath in eventPaths:
    if int(os.path.basename(eventPath)) <= year:
        if int(os.path.basename(eventPath)) == year or wsw_full_refresh:
            event = Event(eventPath)
            event.processEvent()

Processing 2010...
125 entrants loaded
INFO: 270 runs by 42 recognised participants down course H1 on 20101016 - 11:51:23 to 15:27:25
INFO: 27 runs by 9 recognised participants down course H1 on 20101017 - 10:34:09 to 12:06:37
INFO: 13 runs by 2 recognised participants down course H1 on 20101018 - 11:59:49 to 13:53:16
INFO: 205 runs by 38 recognised participants down course S1 on 20101018 - 11:38:06 to 13:28:59
INFO: 352 runs by 49 recognised participants down course S2 on 20101018 - 11:38:28 to 16:29:57
INFO: 5 runs by 2 recognised participants down course H1 on 20101019 - 15:02:09 to 16:14:54
INFO: 221 runs by 42 recognised participants down course S1 on 20101019 - 11:17:01 to 12:59:54
INFO: 673 runs by 69 recognised participants down course S2 on 20101019 - 14:04:46 to 16:59:55
INFO: 4 runs by 2 recognised participants down course S3 on 20101019 - 12:37:25 to 15:24:08
INFO: 25 runs by 7 recognised participants down course H1 on 20101020 - 11:06:23 to 12:55:32
INFO: 5 runs by 1 recog

In [14]:
pc2 = time.perf_counter()
print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Reports completed in 0.63 seconds


## All Done!